# Calibration


   * [3D-cluster: $P_{T}$ resolution](#3D-cluster:-Pt-resolution)
   * [$p_{T}$ response](#$p_{T}$-response)


In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [2]:
# %load ../drawingTools
import ROOT
import math
import uuid

# some useful globals, mainly to deal with ROOT idiosyncrasies
c_idx = 0
p_idx = 0
colors = range(1, 6)
stuff = []
f_idx = 0

ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.30)

ROOT.gStyle.SetCanvasBorderMode(0)
ROOT.gStyle.SetCanvasColor(0)
ROOT.gStyle.SetCanvasDefH(600)
ROOT.gStyle.SetCanvasDefW(800)

# define some utility functions
def newCanvas(name=None, title=None, xdiv=0, ydiv=0, form=4):
    global c_idx
    if name is None:
        name = 'c_{}'.format(uuid.uuid4().hex[:6])
        c_idx += 1
    if title is None:
        title = name
    # print name, title
    canvas = ROOT.TCanvas(name, title)
    if(xdiv*ydiv != 0):
        canvas.Divide(xdiv, ydiv)
    global stuff
    stuff.append(canvas)
    return canvas


def draw(plot, options='', text=None):
    c = newCanvas()
    c.cd()
    plot.Draw(options)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()

    return


def getLegend(x1=0.7, y1=0.71, x2=0.95, y2=0.85):
    global stuff
    legend = ROOT.TLegend(x1, y1, x2, y2)
    stuff.append(legend)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.05)
    return legend


def drawAndProfileX(plot2d, miny=None, maxy=None, do_profile=True, options='', text=None):
    global p_idx
    if miny and maxy:
        plot2d.GetYaxis().SetRangeUser(miny, maxy)
    c = newCanvas()
    c.SetGrid(1, 1)
    c.cd()
    plot2d.Draw(options)
    ROOT.gPad.SetGrid(1, 1)
    ROOT.gStyle.SetGridColor(15)

    if do_profile:
        profname = plot2d.GetName()+'_prof_'+str(p_idx)
        p_idx += 1
        firstbin = 1
        lastbin = -1
        prof = plot2d.ProfileX(profname, firstbin, lastbin, 's')
        prof.SetMarkerColor(2)
        prof.SetLineColor(2)
        prof.Draw('same')

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()


def getText(text, ndc_x, ndc_y):
    global stuff
    rtext = ROOT.TLatex(ndc_x, ndc_y, text)
    stuff.append(rtext)
    rtext.SetNDC(True)
    # rtext.SetTextFont(40)
    rtext.SetTextSize(0.03)
    return rtext


def drawSame(histograms,
             labels,
             options='',
             norm=False,
             logy=False,
             min_y=None,
             max_y=None,
             text=None,
             y_axis_label=None,
             x_axis_label=None,
             v_lines=None,
             h_lines=None):
    global colors
    global stuff
    c = newCanvas(title=histograms[0].GetName())
    c.cd()
    leg = getLegend()

    max_value = max_y
    min_value = min_y
    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in histograms])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in histograms])*1.2

    for hidx, hist in enumerate(histograms):
        hist.SetLineColor(colors[hidx])
        hist.SetStats(False)
        if norm:
            hist.DrawNormalized('same'+','+options, 1.)
        else:
            if hidx:
                hist.Draw('same'+','+options)
            else:
                hist.Draw(options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    histograms[0].GetYaxis().SetRangeUser(min_value, max_value)
    if y_axis_label:
        histograms[0].GetYaxis().SetTitle(y_axis_label)
    if x_axis_label:
            histograms[0].GetXaxis().SetTitle(x_axis_label)

    leg.Draw()
    c.Draw()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    if logy:
        c.SetLogy()

    if v_lines:
        for v_line_x in v_lines:
            aline = ROOT.TLine(v_line_x, c.GetUymin(), v_line_x,  c.GetUymax())
            aline.SetLineStyle(2)
            aline.Draw("same")
            stuff.append(aline)
    if h_lines:
        for h_line_y in h_lines:
            aline = ROOT.TLine(c.GetUxmin(), h_line_y, c.GetUxmax(),  h_line_y)
            aline.SetLineStyle(2)
            aline.Draw("same")
            stuff.append(aline)
    c.Update()


def drawProfileX(histograms, labels, options=''):
    profiles = [hist.ProfileX() for hist in histograms]
    drawSame(profiles, labels, options)


def drawSeveral(histograms, labels, options='', do_profile=False, miny=None, maxy=None, text=None):
    ydiv = int(math.ceil(float(len(histograms))/2))
    for hidx in range(0, len(histograms)):
        newtext = labels[hidx]
        if text:
            newtext = '{}: {}'.format(labels[hidx], text)
        if do_profile:
            drawAndProfileX(histograms[hidx], miny=miny, maxy=maxy, options=options, do_profile=do_profile, text=newtext)
        else:
            draw(histograms[hidx], options=options, text=newtext)


def drawProfileRatio(prof1, prof2, ymin=None, ymax=None, text=None):
    hist1 = prof1.ProjectionX(uuid.uuid4().hex[:6])
    hist2 = prof2.ProjectionX(uuid.uuid4().hex[:6])
    hist1.Divide(hist2)
    draw(hist1)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")

    if ymin is not None and ymax is not None:
        hist1.GetYaxis().SetRangeUser(ymin, ymax)
    ROOT.gPad.Update()


# mean+-nsigmas*RMS.
def drawGaussFit(histo, nsigmas, min, max):
    minfit = histo.GetMean() - nsigmas*histo.GetRMS()
    maxfit = histo.GetMean() + nsigmas*histo.GetRMS()
    drawGFit(histo, min, max, minfit, maxfit)


# Fit a histogram in the range (minfit, maxfit) with a gaussian and
# draw it in the range (min, max)
def drawGFit(histo, min, max, minfit, maxfit):
    # static int i = 0
    # i++
    # gPad->SetGrid(1,1);
    # gStyle->SetGridColor(15);
    histo.GetXaxis().SetRangeUser(min,max)
    global f_idx
    nameF1 = "g{}".format(f_idx)
    f_idx +=1
    g1 = ROOT.TF1(nameF1,"gaus",minfit,maxfit)
    g1.SetLineColor(2)
    g1.SetLineWidth(2)
    histo.Fit(g1,"R")


def drawGraphsSame(histograms, labels, options='', norm=False, logy=False, min_y=None, max_y=None, text=None):
    global colors
    c = newCanvas()
    c.cd()
    leg = getLegend()

    for hidx in range(0, len(histograms)):
        histograms[hidx].SetLineColor(colors[hidx])
        histograms[hidx].Draw('same'+','+options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    max_value = max_y
    min_value = min_y
    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in histograms])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in histograms])*1.2
    histograms[0].GetYaxis().SetRangeUser(min_value, max_value)
    leg.Draw()
    c.Draw()
    if logy:
        c.SetLogy()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    c.Update()


In [3]:
# %load samples.py
import ROOT

version = 'v53'

files = {}
file_keys = {}



class RootFile:
    def __init__(self, file_name):
        global file
        self.file_name = file_name
        if self.file_name not in files.keys():
            print 'get file: {}'.format(self.file_name)
            files[self.file_name] = ROOT.TFile(self.file_name)
        self._file = files[self.file_name]
        self._file_keys = None

    def cd(self):
        self._file.cd()

    def GetListOfKeys(self):
        global file_keys
        if self.file_name not in file_keys.keys():
            print 'get list'
            file_keys[self.file_name] = self._file.GetListOfKeys()
        self._file_keys = file_keys[self.file_name]
        return self._file_keys

class Sample():
    def __init__(cls, name, label, version=None):
        cls.name = name
        cls.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        cls.histo_filename = '../plots1/histos_{}{}.root'.format(cls.name, version)
        cls.histo_file = ROOT.TFile(cls.histo_filename, 'r')#RootFile(cls.histo_filename)


# sample_names = ['ele_flat2to100_PU0',
#                 'ele_flat2to100_PU200',
#                 'photonPt35_PU0',
#                 'photonPt35_PU200']


def get_label_dict(selections):
    dictionary = {}
    for sel in selections:
        dictionary[sel.name] = sel.label
    return dictionary



class HProxy:
    def __init__(self, classtype, tp, tp_sel, gen_sel, root_file):
        self.classtype = classtype
        self.tp = tp
        self.tp_sel = tp_sel
        self.gen_sel = gen_sel
        self.root_file = root_file
        self.instance = None


    def get(self):
        if self.instance is None:
            name = '{}_{}_{}'.format(self.tp, self.tp_sel, self.gen_sel)
            if self.gen_sel == None:
                name = '{}_{}'.format(self.tp, self.tp_sel)
            self.instance = self.classtype(name, self.root_file)
        return self.instance

class HPlot:
    def __init__(self, samples, tp_sets, tp_selections, gen_selections):
        self.tp_sets = tp_sets
        self.tp_selections = tp_selections
        self.gen_selections = gen_selections
        self.pus = []
        for sample in samples:
            self.pus.append(sample.label)
        self.data = pd.DataFrame(columns=['sample', 'pu', 'tp', 'tp_sel', 'gen_sel', 'classtype', 'histo'])
        self.labels_dict = {}

        self.labels_dict.update(tp_sets)
        self.labels_dict.update(tp_selections)
        self.labels_dict.update(gen_selections)
        self.labels_dict.update({'PU0': 'PU0', 'PU200': 'PU200'})

    def cache_histo(self,
                    classtype,
                    samples,
                    pus,
                    tps,
                    tp_sels,
                    gen_sels):
        for sample in samples:
            print sample
            for tp in tps:
                for tp_sel in tp_sels:
                    if gen_sels is None:
                        self.data = self.data.append({'sample': 'ele',
                                                        'pu': sample.label,
                                                        'tp': tp,
                                                        'tp_sel': tp_sel,
                                                        'gen_sel': None,
                                                        'classtype': classtype,
                                                        'histo': HProxy(classtype, tp, tp_sel, None, sample.histo_file)}
                                                          , ignore_index=True)
                    else:
                        for gen_sel in gen_sels:
                            print sample, tp, tp_sel, gen_sel
                            self.data = self.data.append({'sample': 'ele',
                                                            'pu': sample.label,
                                                            'tp': tp,
                                                            'tp_sel': tp_sel,
                                                            'gen_sel': gen_sel,
                                                            'classtype': classtype,
                                                            'histo': HProxy(classtype, tp, tp_sel, gen_sel, sample.histo_file)}
                                                              , ignore_index=True)



    def get_histo(self,
                  classtype,
                  sample=None,
                  pu=None,
                  tp=None,
                  tp_sel=None,
                  gen_sel=None):
        histo = None
        labels = []
        text = ''
        query = '(pu == @pu) & (tp == @tp) & (tp_sel == @tp_sel) & (classtype == @classtype)'
        if gen_sel is not None:
            query += '& (gen_sel == @gen_sel)'
        histo_df = self.data.query(query)

        if histo_df.empty:
            print 'No match found for: pu: {}, tp: {}, tp_sel: {}, gen_sel: {}, classtype: {}'.format(pu, tp, tp_sel, gen_sel, classtype)


        field_counts = histo_df.apply(lambda x: len(x.unique()))
        label_fields = []
        text_fields = []
        # print field_counts
        for field in field_counts.iteritems():
            if(field[1] > 1 and field[0] != 'histo' and field[0] != 'sample'):
                label_fields.append(field[0])
            if(field[1] == 1 and field[0] != 'histo' and field[0] != 'classtype' and field[0] != 'sample'):
                if(gen_sel is None and field[0] == 'gen_sel'):
                    continue
                text_fields.append(field[0])

#         print 'label fields: {}'.format(label_fields)
#         print 'text fields: {}'.format(text_fields)

        for item in histo_df[label_fields].iterrows():
            labels.append(', '.join([self.labels_dict[tx] for tx in item[1].values if self.labels_dict[tx] != '']))

        #print labels
        text = ', '.join([self.labels_dict[fl] for fl in histo_df[text_fields].iloc[0].values if self.labels_dict[fl] != ''])
        histo = [his.get() for his in  histo_df['histo'].values]
        return histo, labels, text



# -------------------------------------------------------------------------
sample_ele_flat2to100_PU0 = Sample('ele_flat2to100_PU0', 'PU0', version)
sample_ele_flat2to100_PU200 = Sample('ele_flat2to100_PU200', 'PU200', version)

samples_ele = [sample_ele_flat2to100_PU0, sample_ele_flat2to100_PU200]

# -------------------------------------------------------------------------

sample_photon_flat8to150_PU0 = Sample('photon_flat8to150_PU0', 'PU0', version)
sample_photon_flat8to150_PU200 = Sample('photon_flat8to150_PU200', 'PU0', version)

samples_photons = [sample_photon_flat8to150_PU0, sample_photon_flat8to150_PU200]


sample_gPt35_PU0 = Sample('photonPt35_PU0', 'Pt35 PU0', version)
sample_gPt35_PU200 = Sample('photonPt35_PU200', 'Pt35 PU200', version)

sample_hadronGun_PU0 = Sample('hadronGun_PU0', 'PU0', version)
sample_hadronGun_PU200 = Sample('hadronGun_PU200', 'PU200', version)


samples_photon = [sample_gPt35_PU0, sample_gPt35_PU200]
samples_hadrons = [sample_hadronGun_PU0, sample_hadronGun_PU200]

sample_nugunrate = Sample('nugun_alleta_pu200', 'PU200', version)
samples_nugunrates = [sample_nugunrate]


from python.selections import tp_rate_selections
from python.selections import tp_match_selections
from python.selections import gen_part_selections

tpsets = {'DEF': 'NNDR',
          'DEFCalib': 'NNDR Calib v1'}

tpset_selections = {}

gen_selections = {}
samples = []


#tpset_selections.update(get_label_dict(tp_rate_selections))
tpset_selections.update(get_label_dict(tp_match_selections))

gen_selections.update(get_label_dict(gen_part_selections))


gen_part_selections: 15


Error in <TFile::TFile>: file ../plots1/histos_photonPt35_PU0_v53.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photonPt35_PU200_v53.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_hadronGun_PU0_v53.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_hadronGun_PU200_v53.root does not exist


## Resolution Studies
### 3D cluster: Pt resolution

In [5]:
%%time

hplot = HPlot(samples, tpsets, tpset_selections, gen_selections)
samples = samples_ele


hplot.cache_histo(classtype=histos.HistoSetReso, 
                  samples=samples,
                  pus=[],
                  tps=tpsets,
                  tp_sels=tpset_selections,
                  gen_sels=gen_selections)



<__main__.Sample instance at 0x11a59fef0>
<__main__.Sample instance at 0x11a59fef0> DEFCalib Em GENEtaB
<__main__.Sample instance at 0x11a59fef0> DEFCalib Em GENEtaABC
<__main__.Sample instance at 0x11a59fef0> DEFCalib Em GENEtaA
<__main__.Sample instance at 0x11a59fef0> DEFCalib Em GENPt40
<__main__.Sample instance at 0x11a59fef0> DEFCalib Em GENEtaC
<__main__.Sample instance at 0x11a59fef0> DEFCalib Em GENEtaAB
<__main__.Sample instance at 0x11a59fef0> DEFCalib Em GENEtaE
<__main__.Sample instance at 0x11a59fef0> DEFCalib Em GENEtaD
<__main__.Sample instance at 0x11a59fef0> DEFCalib Em GENEtaBC
<__main__.Sample instance at 0x11a59fef0> DEFCalib Em GENPt30
<__main__.Sample instance at 0x11a59fef0> DEFCalib Em GENPt20
<__main__.Sample instance at 0x11a59fef0> DEFCalib Em GENEtaBCD
<__main__.Sample instance at 0x11a59fef0> DEFCalib Em GENPt10
<__main__.Sample instance at 0x11a59fef0> DEFCalib Em GENEtaBCDE
<__main__.Sample instance at 0x11a59fef0> DEFCalib Em GEN
<__main__.Sample instan

<__main__.Sample instance at 0x11a59fef0> DEFCalib Pt25 GENPt10
<__main__.Sample instance at 0x11a59fef0> DEFCalib Pt25 GENEtaBCDE
<__main__.Sample instance at 0x11a59fef0> DEFCalib Pt25 GEN
<__main__.Sample instance at 0x11a59fef0> DEFCalib Emv1Pt25 GENEtaB
<__main__.Sample instance at 0x11a59fef0> DEFCalib Emv1Pt25 GENEtaABC
<__main__.Sample instance at 0x11a59fef0> DEFCalib Emv1Pt25 GENEtaA
<__main__.Sample instance at 0x11a59fef0> DEFCalib Emv1Pt25 GENPt40
<__main__.Sample instance at 0x11a59fef0> DEFCalib Emv1Pt25 GENEtaC
<__main__.Sample instance at 0x11a59fef0> DEFCalib Emv1Pt25 GENEtaAB
<__main__.Sample instance at 0x11a59fef0> DEFCalib Emv1Pt25 GENEtaE
<__main__.Sample instance at 0x11a59fef0> DEFCalib Emv1Pt25 GENEtaD
<__main__.Sample instance at 0x11a59fef0> DEFCalib Emv1Pt25 GENEtaBC
<__main__.Sample instance at 0x11a59fef0> DEFCalib Emv1Pt25 GENPt30
<__main__.Sample instance at 0x11a59fef0> DEFCalib Emv1Pt25 GENPt20
<__main__.Sample instance at 0x11a59fef0> DEFCalib Emv1Pt

<__main__.Sample instance at 0x11a59fef0> DEF EmPt30 GENEtaE
<__main__.Sample instance at 0x11a59fef0> DEF EmPt30 GENEtaD
<__main__.Sample instance at 0x11a59fef0> DEF EmPt30 GENEtaBC
<__main__.Sample instance at 0x11a59fef0> DEF EmPt30 GENPt30
<__main__.Sample instance at 0x11a59fef0> DEF EmPt30 GENPt20
<__main__.Sample instance at 0x11a59fef0> DEF EmPt30 GENEtaBCD
<__main__.Sample instance at 0x11a59fef0> DEF EmPt30 GENPt10
<__main__.Sample instance at 0x11a59fef0> DEF EmPt30 GENEtaBCDE
<__main__.Sample instance at 0x11a59fef0> DEF EmPt30 GEN
<__main__.Sample instance at 0x11a59fef0> DEF EmPt20 GENEtaB
<__main__.Sample instance at 0x11a59fef0> DEF EmPt20 GENEtaABC
<__main__.Sample instance at 0x11a59fef0> DEF EmPt20 GENEtaA
<__main__.Sample instance at 0x11a59fef0> DEF EmPt20 GENPt40
<__main__.Sample instance at 0x11a59fef0> DEF EmPt20 GENEtaC
<__main__.Sample instance at 0x11a59fef0> DEF EmPt20 GENEtaAB
<__main__.Sample instance at 0x11a59fef0> DEF EmPt20 GENEtaE
<__main__.Sample in

<__main__.Sample instance at 0x11a59fef0> DEF Emv1 GENPt30
<__main__.Sample instance at 0x11a59fef0> DEF Emv1 GENPt20
<__main__.Sample instance at 0x11a59fef0> DEF Emv1 GENEtaBCD
<__main__.Sample instance at 0x11a59fef0> DEF Emv1 GENPt10
<__main__.Sample instance at 0x11a59fef0> DEF Emv1 GENEtaBCDE
<__main__.Sample instance at 0x11a59fef0> DEF Emv1 GEN
<__main__.Sample instance at 0x11a59ffc8>
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Em GENEtaB
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Em GENEtaABC
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Em GENEtaA
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Em GENPt40
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Em GENEtaC
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Em GENEtaAB
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Em GENEtaE
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Em GENEtaD
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Em GENEtaBC
<__main__.Sample instance at 0x11a59ffc8>

<__main__.Sample instance at 0x11a59ffc8> DEFCalib Emv1Pt25 GENEtaAB
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Emv1Pt25 GENEtaE
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Emv1Pt25 GENEtaD
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Emv1Pt25 GENEtaBC
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Emv1Pt25 GENPt30
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Emv1Pt25 GENPt20
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Emv1Pt25 GENEtaBCD
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Emv1Pt25 GENPt10
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Emv1Pt25 GENEtaBCDE
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Emv1Pt25 GEN
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Pt20 GENEtaB
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Pt20 GENEtaABC
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Pt20 GENEtaA
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Pt20 GENPt40
<__main__.Sample instance at 0x11a59ffc8> DEFCalib Pt20 GEN

<__main__.Sample instance at 0x11a59ffc8> DEF EmPt30 GENEtaB
<__main__.Sample instance at 0x11a59ffc8> DEF EmPt30 GENEtaABC
<__main__.Sample instance at 0x11a59ffc8> DEF EmPt30 GENEtaA
<__main__.Sample instance at 0x11a59ffc8> DEF EmPt30 GENPt40
<__main__.Sample instance at 0x11a59ffc8> DEF EmPt30 GENEtaC
<__main__.Sample instance at 0x11a59ffc8> DEF EmPt30 GENEtaAB
<__main__.Sample instance at 0x11a59ffc8> DEF EmPt30 GENEtaE
<__main__.Sample instance at 0x11a59ffc8> DEF EmPt30 GENEtaD
<__main__.Sample instance at 0x11a59ffc8> DEF EmPt30 GENEtaBC
<__main__.Sample instance at 0x11a59ffc8> DEF EmPt30 GENPt30
<__main__.Sample instance at 0x11a59ffc8> DEF EmPt30 GENPt20
<__main__.Sample instance at 0x11a59ffc8> DEF EmPt30 GENEtaBCD
<__main__.Sample instance at 0x11a59ffc8> DEF EmPt30 GENPt10
<__main__.Sample instance at 0x11a59ffc8> DEF EmPt30 GENEtaBCDE
<__main__.Sample instance at 0x11a59ffc8> DEF EmPt30 GEN
<__main__.Sample instance at 0x11a59ffc8> DEF EmPt20 GENEtaB
<__main__.Sample in

<__main__.Sample instance at 0x11a59ffc8> DEF Emv1Pt10 GENEtaC
<__main__.Sample instance at 0x11a59ffc8> DEF Emv1Pt10 GENEtaAB
<__main__.Sample instance at 0x11a59ffc8> DEF Emv1Pt10 GENEtaE
<__main__.Sample instance at 0x11a59ffc8> DEF Emv1Pt10 GENEtaD
<__main__.Sample instance at 0x11a59ffc8> DEF Emv1Pt10 GENEtaBC
<__main__.Sample instance at 0x11a59ffc8> DEF Emv1Pt10 GENPt30
<__main__.Sample instance at 0x11a59ffc8> DEF Emv1Pt10 GENPt20
<__main__.Sample instance at 0x11a59ffc8> DEF Emv1Pt10 GENEtaBCD
<__main__.Sample instance at 0x11a59ffc8> DEF Emv1Pt10 GENPt10
<__main__.Sample instance at 0x11a59ffc8> DEF Emv1Pt10 GENEtaBCDE
<__main__.Sample instance at 0x11a59ffc8> DEF Emv1Pt10 GEN
<__main__.Sample instance at 0x11a59ffc8> DEF Emv1 GENEtaB
<__main__.Sample instance at 0x11a59ffc8> DEF Emv1 GENEtaABC
<__main__.Sample instance at 0x11a59ffc8> DEF Emv1 GENEtaA
<__main__.Sample instance at 0x11a59ffc8> DEF Emv1 GENPt40
<__main__.Sample instance at 0x11a59ffc8> DEF Emv1 GENEtaC
<__main

In [6]:
hplot.data

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,ele,PU0,DEFCalib,Em,GENEtaB,python.l1THistos.HistoSetReso,<__main__.HProxy instance at 0x11a7cb8c0>
1,ele,PU0,DEFCalib,Em,GENEtaABC,python.l1THistos.HistoSetReso,<__main__.HProxy instance at 0x11a804cb0>
2,ele,PU0,DEFCalib,Em,GENEtaA,python.l1THistos.HistoSetReso,<__main__.HProxy instance at 0x11a7cbfc8>
3,ele,PU0,DEFCalib,Em,GENPt40,python.l1THistos.HistoSetReso,<__main__.HProxy instance at 0x119af2878>
4,ele,PU0,DEFCalib,Em,GENEtaC,python.l1THistos.HistoSetReso,<__main__.HProxy instance at 0x11a5a56c8>
5,ele,PU0,DEFCalib,Em,GENEtaAB,python.l1THistos.HistoSetReso,<__main__.HProxy instance at 0x119af20e0>
6,ele,PU0,DEFCalib,Em,GENEtaE,python.l1THistos.HistoSetReso,<__main__.HProxy instance at 0x11a804098>
7,ele,PU0,DEFCalib,Em,GENEtaD,python.l1THistos.HistoSetReso,<__main__.HProxy instance at 0x11a7ff320>
8,ele,PU0,DEFCalib,Em,GENEtaBC,python.l1THistos.HistoSetReso,<__main__.HProxy instance at 0x119af23b0>
9,ele,PU0,DEFCalib,Em,GENPt30,python.l1THistos.HistoSetReso,<__main__.HProxy instance at 0x11a7ff908>


In [7]:
# dir(histo_reso_df.loc['DEF', 'ele'][0])
tps = ['DEF', 'DEFCalib']
tp_select = ['all', 'Em' ,'Emv1']
gen_select = ['GEN', 'GENEtaA', 'GENEtaAB', 'GENEtaABC', 'GENEtaB', 'GENEtaBC', 'GENEtaBCD', 'GENEtaBCDE', 'GENEtaC', 'GENEtaD', 'GENEtaE']


## $p_{T}$ response

In [8]:
help(drawSame)

Help on function drawSame in module __main__:

drawSame(histograms, labels, options='', norm=False, logy=False, min_y=None, max_y=None, text=None, y_axis_label=None, x_axis_label=None, v_lines=None, h_lines=None)



In [9]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.HistoSetReso, 'ele', ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            drawSame([his.hreso.h_ptResp for his in hsets], labels, norm=True, text=text, v_lines=[1.0])


In [10]:
help(drawGaussFit)

Help on function drawGaussFit in module __main__:

drawGaussFit(histo, nsigmas, min, max)
    # mean+-nsigmas*RMS.



In [11]:
class HistoTemplate(object):
    def __init__(self, histo):
        self._histo = histo
        
    def __call__(self, x, par):
        xx = par[1] * (x[0] - par[2])
        xMin = self._histo.GetBinCenter(1)
        xMax = self._histo.GetBinCenter(self._histo.GetNbinsX())
    
        if xx < xMin or xx >= xMax:
            return 1.e-10
        else:
            bin = self._histo.FindBin(xx)
            bin1 = 0
            bin1 = 0
            
            if xx >= self._histo.GetBinCenter(bin):
                bin1 = bin
                bin2 = bin+1
            else:
                bin1 = bin-1
                bin2 = bin
            
            x1 = self._histo.GetBinCenter(bin1)
            y1 = self._histo.GetBinContent(bin1)
            x2 = self._histo.GetBinCenter(bin2)
            y2 = self._histo.GetBinContent(bin2)
            
            m = 1. * (y2 - y1) / (x2 - x1)
            
            if (y1 + m * (xx - x1)) < 1.e-10:
                return 1.e-10
            
            return par[0] * par[1] * (y1 + m * (xx - x1))
        return 1.e-10
            
        

In [12]:
def double_goussian(x, params):
    if x[0]>= params[0]:
        return params[1]*ROOT.TMath.Gaus(x[0], params[0], params[2])
    else:
        return (params[1])*(ROOT.TMath.Gaus(x[0], params[0], params[3])+(params[4]+x[0]*params[5]+params[6]*x[0]*x[0]))
fitf = ROOT.TF1('myfunc', double_goussian, 0, 3, 7)
# fitf.SetParNames('mean', "norm", 'sigma_p', 'sigma_m', 'norm_l')
fitf.SetParameters(1,1,0.1,0.2, 1, 3, 4)
c = newCanvas()
fitf.Draw()
c.Draw()

In [13]:
particle = 'eleC'
tp_set = 'DEF_em'

# pt_bins = [(1, 4), (5, 9), (10, 14), (15, 19), (20, 50)]
pt_bins = [(1, 4), (5, 9), (10, 14), (15, 19), (20, 50)]

hsets, labels, text = hplot.get_histo(histos.HistoSetReso, 'ele', ['PU0', 'PU200'], 'DEF', 'Em', 'GENEtaC') 
hByTPset = [his.hreso.h_ptRespVpt for his in hsets]
histogram = hByTPset[0]



In [14]:
template = HistoTemplate(histogram.ProjectionY(uuid.uuid4().hex[:6], 20, 50))
fitf = ROOT.TF1('myfunc', template, 0, 3, 3)
fitf.SetParNames('norm', 'scale', 'shift')
c = newCanvas()
# fitf.SetParameters(1.,1.,0.)
# fitf.Draw()
fitf.SetParameters(10.,3.,1.5)
fitf.Draw()
c.Draw()    


In [15]:
for pt_bin in pt_bins:
#     project_hist
    project_hist = histogram.ProjectionY(uuid.uuid4().hex[:6], pt_bin[0], pt_bin[1])
    fitf.SetParameters(1.,1.,0.)
#     fitf.FixParameter(2, 0)
    project_hist.Fit('gaus','SE', '',0.9,1.)
    draw(project_hist, options='hist')

 FCN=1.42716e-07 FROM MINOS     STATUS=FAILURE       117 CALLS        1221 TOTAL
                     EDM=3.72522e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     3.78042e+02   1.95170e+03   1.53085e+03   0.00000e+00
   2  Mean         7.75373e-01   5.83170e-01  -7.24279e+00   0.00000e+00
   3  Sigma        9.62504e-02   1.61127e-01   1.61127e-01  -1.72236e+02
 FCN=5.25198e-09 FROM MINOS     STATUS=SUCCESSFUL    167 CALLS         711 TOTAL
                     EDM=1.05009e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.15273e+03   1.45902e+02   1.79522e-01   6.37958e-06
   2  Mean         8.88382e-01   2.68253e-02  -2.28828e-05   8.76444e-02
   3  Sigma        7.0765

In [16]:
from array import array
calibFactors_hist = ROOT.TH2F('calibFactors_hist', 'calib factor', 5, array('d',[0., 10., 20., 30., 40., 100.]), 3, array('d',[1.52, 1.7, 2.4, 2.8]))


In [17]:
#draw(calibFactors_hist)

In [18]:
calibFactors_df = pd.DataFrame(columns=['eta_l', 'eta_h', 'pt_l', 'pt_h', 'calib'])

In [19]:
eta_ranges = {'GENEtaB': (1.52, 1.7),
              'GENEtaC': (1.7, 2.4),
              'GENEtaD': (2.4, 2.8)}

tp_set = 'DEF'
# pt_bins = [(1, 4), (5, 9), (10, 14), (15, 19), (20, 50)]
pt_bins = [(1, 5), (6, 10), (11, 15), (16, 20), (21, 50)]

for id_p,gen_sel in enumerate(['GENEtaB', 'GENEtaC','GENEtaD']):

    hsets, labels, text = hplot.get_histo(histos.HistoSetReso, 'ele', ['PU0', 'PU200'], 'DEF', 'Em', gen_sel) 
    hByTPset = [his.hreso.h_ptRespVpt for his in hsets]
    histogram = hByTPset[0]
    draw(histogram, options='COLZ', text=text)
    for id_pt,pt_bin in enumerate(pt_bins):
        pt_bin_low = histogram.GetXaxis().GetBinLowEdge(pt_bin[0])
        pt_bin_up = histogram.GetXaxis().GetBinUpEdge(pt_bin[1])
        print '==== ETA region: {}, {} < pT < {}'.format(text, pt_bin_low, pt_bin_up)

        project_hist = histogram.ProjectionY(uuid.uuid4().hex[:6], pt_bin[0], pt_bin[1])
        project_hist.GetXaxis().SetRangeUser(0.53, 1.5)
        max_bin = project_hist.GetMaximumBin()
        max_value = project_hist.GetBinCenter(max_bin)
        rms_value = project_hist.GetRMS()
        project_hist.GetXaxis().SetRangeUser(0, 3)
        print '   max_value = {}, RMS = {}'.format(max_value, rms_value)
    
        k_l = 0.5
        k_r = 1.
        if(gen_sel) == 'eleD':
            k_l = 1.5
            k_r = 2.
        #fitf = ROOT.TF1('myfunc', 'gaus')
        #fitf.SetParameter(0,0.93)
        result = project_hist.Fit('gaus','QERLS+', '', max_value-k_l*rms_value,  max_value+k_r*rms_value)
        result.Print()
        print '   mean = {}'.format(result.GetParams()[1])
        
        draw(project_hist, options='hist', text=text)
        calibFactors_hist.SetBinContent(id_pt+1, id_p+1, result.GetParams()[1])
        calibFactors_df= calibFactors_df.append({'eta_l': eta_ranges[gen_sel][0], 
                                                 'eta_h': eta_ranges[gen_sel][1], 
                                                 'pt_l': pt_bin_low, 
                                                 'pt_h': pt_bin_up, 
                                                 'calib': result.GetParams()[1]}, ignore_index=True)
#     bin_min = histogram.GetBinWithContent(pt_bin[0])
#     bin_max = histogram.GetBinWithContent(pt_bin[1])-1
#     print 'pt_bins: {} bin # ({}, {})'.format(pt_bin, bin_min, bin_max)



# #drawGaussFit(hByTPset[0], 3, 0, 3)
# draw(hByTPset[0])
# fitf = ROOT.TF1('myfunc', 'crystalball')
# fitf.SetParameter(1,0.93)
# #fitf.SetParameter('Constant', hByTPset[0].Integral())

# # fitf.SetParameter('Mean', 0.93)
# # fitf.SetParameter('Sigma', hByTPset[0].GetRMS())

# #fitf.SetParameter(2, 5)

# hByTPset[0].Fit('myfunc','VLE', '', 0, 0.94 )
# print hByTPset[0].GetName()
# print hByTPset[0].GetEntries()

==== ETA region: NNDR, EGId, 1.52 < |#eta^{GEN}| <= 1.7, 0.0 < pT < 10.0
   max_value = 0.885, RMS = 0.139516473545
   mean = 0.879161251383
==== ETA region: NNDR, EGId, 1.52 < |#eta^{GEN}| <= 1.7, 10.0 < pT < 20.0
   max_value = 0.885, RMS = 0.147195689314
   mean = 0.872851017251
==== ETA region: NNDR, EGId, 1.52 < |#eta^{GEN}| <= 1.7, 20.0 < pT < 30.0
   max_value = 0.975, RMS = 0.146290475022
   mean = 0.947657618054
==== ETA region: NNDR, EGId, 1.52 < |#eta^{GEN}| <= 1.7, 30.0 < pT < 40.0
   max_value = 0.945, RMS = 0.136410496009
   mean = 0.948857501931
==== ETA region: NNDR, EGId, 1.52 < |#eta^{GEN}| <= 1.7, 40.0 < pT < 100.0
   max_value = 0.975, RMS = 0.105880253616
   mean = 0.97032047135
==== ETA region: NNDR, EGId, 1.7 < |#eta^{GEN}| <= 2.4, 0.0 < pT < 10.0
   max_value = 0.855, RMS = 0.126721853489
   mean = 0.857007129826
==== ETA region: NNDR, EGId, 1.7 < |#eta^{GEN}| <= 2.4, 10.0 < pT < 20.0
   max_value = 0.915, RMS = 0.125671345646
   mean = 0.907993923528
==== ETA r

In [22]:

draw(calibFactors_hist, options='COLZ')


In [23]:
calibFactors_df

,eta_l,eta_h,pt_l,pt_h,calib
0,1.52,1.7,0.0,10.0,0.879161
1,1.52,1.7,10.0,20.0,0.872851
2,1.52,1.7,20.0,30.0,0.947658
3,1.52,1.7,30.0,40.0,0.948858
4,1.52,1.7,40.0,100.0,0.970320
5,1.70,2.4,0.0,10.0,0.857007
6,1.70,2.4,10.0,20.0,0.907994
7,1.70,2.4,20.0,30.0,0.937690
8,1.70,2.4,30.0,40.0,0.943678
9,1.70,2.4,40.0,100.0,0.962269


In [20]:
with open('calib_v53_v0.json', 'w') as f:
    f.write(calibFactors_df.to_json(orient='split'))

In [32]:
calibFactors_reco_df = pd.DataFrame(columns=['eta_l', 'eta_h', 'pt_l', 'pt_h', 'calib'])


hsets, labels, text = hplot.get_histo(histos.HistoSetReso, 'ele', ['PU0', 'PU200'], 'DEF', 'Em', 'GEN') 
hByTPset = [his.hreso.h_ptRespVetaVptL1 for his in hsets]
drawSeveral(hByTPset, labels=labels)
histo_3d = hByTPset[0]
#         for eta_range in [(7,7), (8,11), (12, 14)]:
for eta_range in [(7,7), (8,9), (10,11), (12, 14), (15,18)]:
    eta_range_values = (histo_3d.GetXaxis().GetBinLowEdge(eta_range[0]), histo_3d.GetXaxis().GetBinUpEdge(eta_range[1]))
    # for pt_range in [(3,5), (6,10), (11,15), (16,20), (21,50)]:
    for pt_range in [(3,5), (6,7), (8,9), (10,11), (12,13), (14,15), (16,17), (18,19), (20,21), (22,50)]:
        pt_range_values = (histo_3d.GetYaxis().GetBinLowEdge(pt_range[0]), histo_3d.GetYaxis().GetBinUpEdge(pt_range[1]))
        print 'eta range: from eta {} (bin {}) to eta {} (bin {})'.format(eta_range_values[0], eta_range[0], eta_range_values[1], eta_range[1])
        print 'pt range: from pt {} [GeV] (bin {}) to pt {} [GeV] (bin {})'.format(pt_range_values[0], pt_range[0], pt_range_values[1], pt_range[1])
        hname = 'pt{}to{}_eta{}to{}'.format(pt_range_values[0], pt_range_values[1], eta_range_values[0], eta_range_values[1])
        respPlot = histo_3d.ProjectionZ(hname, eta_range[0], eta_range[1], pt_range[0], pt_range[1], '')

        respPlot.GetXaxis().SetRangeUser(0.7, 1.5)
        max_bin = respPlot.GetMaximumBin()
        max_value = respPlot.GetBinCenter(max_bin)
        rms_value = respPlot.GetRMS()
        respPlot.GetXaxis().SetRangeUser(0, 3)
        print '   max_value = {}, RMS = {}'.format(max_value, rms_value)

        k_l = 0.5
        k_r = 1.
        if pt_range_values[0] > 30 or eta_range[0] > 14:
            k_l = 1.
        #fitf = ROOT.TF1('myfunc', 'gaus')
        #fitf.SetParameter(0,0.93)
        result = respPlot.Fit('gaus','QERLS+', '', max_value-k_l*rms_value,  max_value+k_r*rms_value)
        result.Print()
        print '   mean = {}'.format(result.GetParams()[1])

        calibFactors_reco_df= calibFactors_reco_df.append({'eta_l': eta_range_values[0], 
                                                           'eta_h': eta_range_values[1], 
                                                           'pt_l': pt_range_values[0],
                                                           'pt_h': pt_range_values[1],
                                                           'calib': result.GetParams()[1]}, ignore_index=True)
        drawSame([respPlot], labels=['PU0'], text=hname)
        ROOT.gStyle.SetOptFit(11111)


eta range: from eta 1.6 (bin 7) to eta 1.7 (bin 7)
pt range: from pt 4.0 [GeV] (bin 3) to pt 10.0 [GeV] (bin 5)
   max_value = 0.765, RMS = 0.084635362277
   mean = 0.778916107392
eta range: from eta 1.6 (bin 7) to eta 1.7 (bin 7)
pt range: from pt 10.0 [GeV] (bin 6) to pt 14.0 [GeV] (bin 7)
   max_value = 0.735, RMS = 0.0971112778091
   mean = 0.820097352315
eta range: from eta 1.6 (bin 7) to eta 1.7 (bin 7)
pt range: from pt 14.0 [GeV] (bin 8) to pt 18.0 [GeV] (bin 9)
   max_value = 0.855, RMS = 0.0951861080879
   mean = 0.895717666585
eta range: from eta 1.6 (bin 7) to eta 1.7 (bin 7)
pt range: from pt 18.0 [GeV] (bin 10) to pt 22.0 [GeV] (bin 11)
   max_value = 0.945, RMS = 0.0967764512117
   mean = 0.938889371288
eta range: from eta 1.6 (bin 7) to eta 1.7 (bin 7)
pt range: from pt 22.0 [GeV] (bin 12) to pt 26.0 [GeV] (bin 13)
   max_value = 0.885, RMS = 0.0932133979701
   mean = 0.931578168233
eta range: from eta 1.6 (bin 7) to eta 1.7 (bin 7)
pt range: from pt 26.0 [GeV] (bin 14)

eta range: from eta 2.4 (bin 15) to eta 2.8 (bin 18)
pt range: from pt 42.0 [GeV] (bin 22) to pt 100.0 [GeV] (bin 50)
   max_value = 0.945, RMS = 0.0484130607385
   mean = 0.958354293521

****************************************
Minimizer is Minuit / Migrad
MinFCN                    =     0.519465
Chi2                      =      1.05035
NDf                       =            1
Edm                       =  1.79815e-07
NCalls                    =          125
Constant                  =      33.8432   +/-   4.72068     
Mean                      =     0.778916   +/-   0.0127581   
Sigma                     =    0.0669667   +/-   0.0303447    	 (limited)

****************************************
Minimizer is Minuit / Migrad
MinFCN                    =      1.16679
Chi2                      =      2.59754
NDf                       =            2
Edm                       =  4.24258e-09
NCalls                    =          221
Constant                  =      34.6401   +/-   4.33264     
M

In [33]:
calibFactors_reco_df

,eta_l,eta_h,pt_l,pt_h,calib
0,1.6,1.7,4.0,10.0,0.778916
1,1.6,1.7,10.0,14.0,0.820097
2,1.6,1.7,14.0,18.0,0.895718
3,1.6,1.7,18.0,22.0,0.938889
4,1.6,1.7,22.0,26.0,0.931578
5,1.6,1.7,26.0,30.0,0.971356
6,1.6,1.7,30.0,34.0,0.939163
7,1.6,1.7,34.0,38.0,0.957731
8,1.6,1.7,38.0,42.0,0.942559
9,1.6,1.7,42.0,100.0,0.967016


In [34]:
with open('calib__v53_v2.json', 'w') as f:
    f.write(calibFactors_reco_df.to_json())

## read and plot the calibration factors

In [35]:
import pandas as pd
calibFactors_reco_df = pd.read_json('calib__v53_v2.json')

In [36]:
calibFactors_reco_df


,calib,eta_h,eta_l,pt_h,pt_l
0,0.778916,1.7,1.6,10,4
1,0.820097,1.7,1.6,14,10
10,0.846952,1.9,1.7,10,4
11,0.887712,1.9,1.7,14,10
12,0.906317,1.9,1.7,18,14
13,0.943263,1.9,1.7,22,18
14,0.939927,1.9,1.7,26,22
15,0.942948,1.9,1.7,30,26
16,0.949723,1.9,1.7,34,30
17,0.944230,1.9,1.7,38,34


In [37]:
import numpy as np

bins_eta = np.sort(calibFactors_reco_df['eta_l'].unique())
bins_eta = np.append(bins_eta, np.sort(calibFactors_reco_df['eta_h'].unique())[-1])
bins_eta

array([ 1.6,  1.7,  1.9,  2.1,  2.4,  2.8])

In [38]:
bins_pt = np.sort(calibFactors_reco_df['pt_l'].unique())
bins_pt = np.append(bins_pt, np.sort(calibFactors_reco_df['pt_h'].unique())[-1])
bins_pt

array([  4,  10,  14,  18,  22,  26,  30,  34,  38,  42, 100])

In [46]:

print len(bins_eta)-1
print len(bins_pt)-1
from array import array
calibFactors_hist = ROOT.TH2F('calibFactors_hist', 
                              'calib factor (E_{L1}/E_{GEN}); #eta^{L1}; p_{T}^{L1} [GeV];', 
                              len(bins_eta)-1, array('d',bins_eta), 
                              len(bins_pt)-1,array('d', bins_pt))


calibFactors_hist

5
10


<ROOT.TH2F object ("calibFactors_hist") at 0x7fc5a0949400>

Warning in <TFile::Append>: Replacing existing TH1: calibFactors_hist (Potential memory leak).


In [47]:

for bin_eta_id,eta_bin in enumerate(bins_eta[:-1]):
    for bin_pt_id,pt_bin in enumerate(bins_pt[:-1]):
        corr = calibFactors_reco_df[(calibFactors_reco_df.eta_l <= eta_bin) & (calibFactors_reco_df.eta_h > eta_bin) & (calibFactors_reco_df.pt_l <= pt_bin) & (calibFactors_reco_df.pt_h > pt_bin)]
        #print corr
        print 'ix: {}, x: {}, iy: {}, y: {}, value: {}'.format(bin_eta_id,eta_bin, bin_pt_id, pt_bin, corr['calib'].iloc[0])
        calibFactors_hist.SetBinContent(bin_eta_id+1, bin_pt_id+1, corr['calib'].iloc[0])

ix: 0, x: 1.6, iy: 0, y: 4, value: 0.7789161074
ix: 0, x: 1.6, iy: 1, y: 10, value: 0.8200973523
ix: 0, x: 1.6, iy: 2, y: 14, value: 0.8957176666
ix: 0, x: 1.6, iy: 3, y: 18, value: 0.9388893713
ix: 0, x: 1.6, iy: 4, y: 22, value: 0.9315781682
ix: 0, x: 1.6, iy: 5, y: 26, value: 0.9713559332
ix: 0, x: 1.6, iy: 6, y: 30, value: 0.9391629345
ix: 0, x: 1.6, iy: 7, y: 34, value: 0.9577314686
ix: 0, x: 1.6, iy: 8, y: 38, value: 0.9425587151
ix: 0, x: 1.6, iy: 9, y: 42, value: 0.9670160164
ix: 1, x: 1.7, iy: 0, y: 4, value: 0.8469515512
ix: 1, x: 1.7, iy: 1, y: 10, value: 0.8877118706
ix: 1, x: 1.7, iy: 2, y: 14, value: 0.906317489
ix: 1, x: 1.7, iy: 3, y: 18, value: 0.9432625544
ix: 1, x: 1.7, iy: 4, y: 22, value: 0.9399265537
ix: 1, x: 1.7, iy: 5, y: 26, value: 0.9429478773
ix: 1, x: 1.7, iy: 6, y: 30, value: 0.9497225248
ix: 1, x: 1.7, iy: 7, y: 34, value: 0.9442299674
ix: 1, x: 1.7, iy: 8, y: 38, value: 0.9422303213
ix: 1, x: 1.7, iy: 9, y: 42, value: 0.9607127351
ix: 2, x: 1.9, iy: 0, y

In [48]:
bins_eta[:-1]

array([ 1.6,  1.7,  1.9,  2.1,  2.4])

In [49]:
draw(calibFactors_hist, options='COLZ')
# calibFactors_hist.Dump()

In [ ]:
for particle in ['eleA','eleAAA', 'eleAA']:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_ptResp for histo in histo_reso_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=False, text=titles_df.loc[tp_set, particle], v_lines=[1.0])


In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_ptRespVpt for histo in histo_reso_df.loc[tp_set, particle]]
        drawSeveral(hByTPset, labels, options="COLZ", do_profile=True, text=titles_df.loc[tp_set, particle])


In [ ]:
for particle in particles:
    for tp_set in tp_sets:
        hByTPset = [histo.h_ptResVpt.ProjectionY(uuid.uuid4().hex[:6], 6, 10) for histo in histo_reso_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle]+' and 10 < p_{t} < 20 GeV')


In [ ]:
for particle in ['ele']:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_ptRespVeta for histo in histo_reso_df.loc[tp_set, particle]]
        drawSeveral(hByTPset, labels, options="COLZ", do_profile=True, text=titles_df.loc[tp_set, particle])


In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_ptResp for histo in histo_resoCone_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle], v_lines=[1.0])


In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_ptRespVpt for histo in histo_resoCone_df.loc[tp_set, particle]]
        drawSeveral(hByTPset, labels, options="COLZ", do_profile=True, text=titles_df.loc[tp_set, particle])


#### All clusters in DR 0.2 from GEN

In [ ]:
histo_resoCone_df = pd.DataFrame(index=tp_sets, columns=particles)
for tp_set in tp_sets:
    for particle in particles:
        histo_resoCone_df.loc[tp_set][particle] =  [histos.HistoSetReso('{}_{}'.format(tp_set, particle), sample.histo_file).hresoCone for sample in samples]
histo_resoCone_df

In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_ptRes for histo in histo_resoCone_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])


In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_ptResp for histo in histo_resoCone_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])


In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_n010 for histo in histo_resoCone_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])


In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_n025 for histo in histo_resoCone_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])


We now compare the 2 defitnitions (highest pt and all clusters in cone) for the same PU point

In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset_best = [histo.h_ptRes for histo in histo_reso_df.loc[tp_set, particle]]
        hByTPset_cone = [histo.h_ptRes for histo in histo_resoCone_df.loc[tp_set, particle]]

        drawSame([hByTPset_best[0], hByTPset_cone[0]], ['PU0 best', 'PU0 cone'], norm=True, text=titles_df.loc[tp_set, particle])
        drawSame([hByTPset_best[1], hByTPset_cone[1]], ['PU200 best', 'PU200 cone'], norm=True, text=titles_df.loc[tp_set, particle])


We now show the behavior of the pt residuals vs pt of the GEN particle

In [ ]:
for algo in algos:
    hByAlgo = [histo.h_ptResVpt for histo in histosAlgos_all.loc[algo].hresoCone]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSeveral(hByAlgo, histosAlgos_all.loc[algo]['labels'], 'COLZ',do_profile=True)
    #print hByAlgo[0].GetName()

In [ ]:
for algo in algos:
    hCl3D_ptResVpt_0 = [histo.h_ptResVpt.ProjectionY(uuid.uuid4().hex[:6], 1, 5) for histo in histosAlgos_all.loc[algo].hresoCone]
    for histo in hCl3D_ptResVpt_0:
        histo.SetTitle('3D cluster reso (GeV) for 1 < pt < 10 GeV')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hCl3D_ptResVpt_0, histosAlgos_all.loc[algo]['labels'], norm=True)


In [ ]:
for algo in algos:
    hCl3D_ptResVpt_1 = [histo.h_ptResVpt.ProjectionY(uuid.uuid4().hex[:6], 6, 10) for histo in histosAlgos_all.loc[algo].hresoCone]
    for histo in hCl3D_ptResVpt_1:
        histo.SetTitle('3D cluster reso (GeV) for 10 < pt < 20 GeV')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hCl3D_ptResVpt_1, histosAlgos_all.loc[algo]['labels'], norm=True)


In [ ]:
for algo in algos:
    hCl3D_ptResVpt_2 = [histo.h_ptResVpt.ProjectionY(uuid.uuid4().hex[:6], 11, 15) for histo in histosAlgos_all.loc[algo].hresoCone]
    for histo in hCl3D_ptResVpt_2:
        histo.SetTitle('3D cluster reso (GeV) for 20 < pt < 30 GeV')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hCl3D_ptResVpt_2, histosAlgos_all.loc[algo]['labels'], norm=True)


In [ ]:
for algo in algos:
    hCl3D_ptResVpt_3 = [histo.h_ptResVpt.ProjectionY(uuid.uuid4().hex[:6], 16, 20) for histo in histosAlgos_all.loc[algo].hresoCone]
    for histo in hCl3D_ptResVpt_3:
        histo.SetTitle('3D cluster reso (GeV) for 30 < pt < 40 GeV')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hCl3D_ptResVpt_3, histosAlgos_all.loc[algo]['labels'], norm=True)


In [ ]:
for algo in algos:
    hByAlgo = [histo.h_ptResVeta for histo in histosAlgos_all.loc[algo].hresoCone]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSeveral(hByAlgo, histosAlgos_all.loc[algo]['labels'], 'COLZ',do_profile=True)

In [ ]:
for algo in algos:
    hByAlgo = [histo.h_ptResVnclu for histo in histosAlgos_all.loc[algo].hresoCone]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSeveral(hByAlgo, histosAlgos_all.loc[algo]['labels'], 'COLZ',do_profile=True)

### Energy resolution


In [ ]:


for algo in algos:
    hByAlgo = [histo.h_energyRes for histo in histosAlgos_all.loc[algo].hreso]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSame(hByAlgo, histosAlgos_all.loc[algo]['labels'], norm=True)



In [ ]:

for s_id in range(0, len(samples)):
    labels_bySample = histosAlgos_all['labels'].apply(lambda x: x[s_id])
    hBySample = [histo.h_energyRes for histo in  histosAlgos_all['hreso'].apply(lambda x: x[s_id])]
    drawSame(hBySample, labels_bySample,'hist',logy=True)


## Postion resolution of 2D clusters

In [ ]:
for algo in algos:
    hByAlgo = [histo.h_xResVlayer for histo in histosAlgos_all.loc[algo].hreso2D]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSeveral(hByAlgo, histosAlgos_all.loc[algo]['labels'], 'COLZ',do_profile=True, miny=-2, maxy=2)

In [ ]:
for algo in algos:
    hByAlgo = [histo.h_xResVlayer.ProjectionY(uuid.uuid4().hex[:6], 1, 3) for histo in histosAlgos_all.loc[algo].hreso2D]
    for histo in hByAlgo:
        histo.SetTitle('2D cluster x reso (cm) for layer 1 and 3')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hByAlgo, histosAlgos_all.loc[algo]['labels'], norm=True, options='hist')


In [ ]:
for algo in algos:
    hByAlgo = [histo.h_xResVlayer.ProjectionY(uuid.uuid4().hex[:6], 12, 15) for histo in histosAlgos_all.loc[algo].hreso2D]
    for histo in hByAlgo:
        histo.SetTitle('2D cluster x reso (cm) for layer 13 and 15')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hByAlgo, histosAlgos_all.loc[algo]['labels'], norm=True, options='hist')


In [ ]:
for algo in algos:
    hByAlgo = [histo.h_yResVlayer for histo in histosAlgos_all.loc[algo].hreso2D]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSeveral(hByAlgo, histosAlgos_all.loc[algo]['labels'], 'COLZ',do_profile=True, miny=-2, maxy=2)